# Location Selection

Now, we need to use the data we obtained to help make a decision on where to locate the business. For this example, we will explore two possible scenarios:

- Casual dining Thai restaurant targeting upper-middle class families and working professionals
- Pub targeting college/university students and young working professionals

A metric needs to be created so each potential venue can be graded on multiple factors - level of competition, size of potential market, and investment required.

In [1]:
import folium
import geopandas as gpd
import h3
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.preprocessing import MaxAbsScaler

In [2]:
df = gpd.read_feather('../data/bangalore_clustered.feather')
pd.options.display.max_colwidth = 20
df.head()

,id,cluster,address,geometry,pop_total,cost_sqft,ATM,Arts & Entertainment,Asian Restaurant,Athletics & Sports,...,Quick Bites,Residence,Restaurant,Salon,School,Shop & Service,Shopping Mall,Spiritual Center,Travel & Transport,Vegetarian / Vegan Restaurant
0,8860169669fffff,3,"Yelahanka Sante,...",POLYGON ((77.605...,1186.041718,5135.624541,0,0,1,0,...,1,0,4,1,0,6,0,2,0,2
1,8860169661fffff,1,"Yelahanka, Kempe...",POLYGON ((77.600...,1845.480809,5140.663527,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1
2,8860169753fffff,1,"Century Artizan,...",POLYGON ((77.597...,1786.194897,5038.134713,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
3,8861892db3fffff,1,"Yelahanka, Kempe...",POLYGON ((77.613...,1413.219880,4896.586595,0,1,0,2,...,0,4,1,0,0,2,0,2,0,0
4,8860169645fffff,2,"Bellary Road, Am...",POLYGON ((77.600...,2635.116823,5134.151371,0,2,3,1,...,0,0,1,0,0,9,1,0,0,0


In [3]:
scaler = MaxAbsScaler()
feature_data = df.drop(columns = ['id', 'cluster', 'geometry', 'address', ])
scaled_features = scaler.fit_transform(feature_data)

df_features = pd.DataFrame(scaled_features, columns = feature_data.columns, index = df['id'])

df_features.describe()

,pop_total,cost_sqft,ATM,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,Bakery & Dessert,Bank,Cafeteria,...,Quick Bites,Residence,Restaurant,Salon,School,Shop & Service,Shopping Mall,Spiritual Center,Travel & Transport,Vegetarian / Vegan Restaurant
count,875.000000,875.000000,875.000000,875.000000,875.000000,875.000000,875.000000,875.000000,875.000000,875.000000,...,875.000000,875.000000,875.000000,875.000000,875.000000,875.000000,875.000000,875.000000,875.000000,875.000000
mean,0.122414,0.440091,0.032571,0.084800,0.086000,0.103510,0.066032,0.101143,0.130514,0.020408,...,0.068571,0.134122,0.145016,0.109714,0.081143,0.122198,0.023143,0.081306,0.015429,0.036857
std,0.151827,0.107075,0.132403,0.154519,0.163857,0.146459,0.126171,0.146415,0.196116,0.072183,...,0.128790,0.165991,0.181010,0.188992,0.176538,0.157583,0.091666,0.138084,0.092892,0.108288
min,0.005117,0.281397,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.028477,0.367507,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.049906,0.407190,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.071429,0.111111,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000
75%,0.166456,0.487180,0.000000,0.200000,0.250000,0.142857,0.111111,0.125000,0.200000,0.000000,...,0.166667,0.214286,0.222222,0.250000,0.000000,0.153846,0.000000,0.142857,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


We will use a weighted sum of selected features to create a score for each location. Complementary venues should increase the score and competition or other detractors should reduce the score.

A function is defined to calculate a location's score. An option has 

In [4]:
def scoreLocation(location, weights = None):
    # Equal weights for all features unless specified
    if not weights:
        cols = location.index
        n = len(cols)
        wt = [1/n for i in range(n)]
        weights = dict(zip(cols, wt))
        
    score = 0 # Initialize score
    for col, weight in weights.items():
        score += location[col] * weight
    
    return score

## Scenario 1: Casual Dining

For this type of restaurant, our primary competition is other similar restaurants, plus fast food or snack joints to some extent. Complementary venues would include residential areas, office locations, shopping malls and movie theaters - these should generally be good indicators of high footfall.

In [5]:
# Assign relative importance of different venues
weights = {
    'pop_total': 10,
    'cost_sqft': -15,
    'Asian Restaurant': -10,
    'Indian Restaurant': -7,
    'Restaurant': -7,
    'Vegetarian / Vegan Restaurant': -5,
    'Quick Bites': -3,
    'Fast Food': -3,
    'Residence': 15,
    'Office': 12,
    'Shopping Mall': 10,
    'Movie Theater': 10
}

In [6]:
scores = pd.Series(dtype = 'float')
counts = []
# For each cell, we will add the cells own score plus 20% of the score of neighbouring cells.
for id, row in df_features.iterrows():
    score = scoreLocation(row, weights)
    neighbours = list(h3.k_ring(id, 1))
    neighbours.remove(id) # Remove root cell - no double counting
    count = 0 # Number of neighbours (check)
    for n in neighbours:
        if n in df_features.index:
            score += (0.2 * scoreLocation(df_features.loc[n], weights))
            count +=1
    scores[id] = score
    counts.append(count)

df_scores_1 = df.copy().set_index('id', drop = False)
df_scores_1.insert(2, 'score', scores)
df_scores_1 = df_scores_1.sort_values('score', ascending = False)

# Display top 10 locations
pd.options.display.max_colwidth = -1
df_scores_1[['id', 'cluster', 'score', 'address']].head(10)

<ipython-input-6-d5e44c537046>:21: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.options.display.max_colwidth = -1


,id,cluster,score,address
id,,,,
8861892461fffff,8861892461fffff,0,7.274146,"Mangammanapalya, Bommanahalli Zone, Bengaluru, Bangalore South, Bangalore Urban, Karnataka, 560068, India"
8861892463fffff,8861892463fffff,1,3.529370,"Mangammanapalya, Bommanahalli Zone, Bengaluru, Bangalore South, Bangalore Urban, Karnataka, 560068, India"
8861892439fffff,8861892439fffff,0,2.960473,"Bommanahalli Ward, Bommanahalli Zone, Bengaluru, Bangalore South, Bangalore Urban, Karnataka, 76, India"
8861892eb1fffff,8861892eb1fffff,0,2.493221,"Richards Town, Sagayarapuram Ward, East Zone, Bengaluru, Bangalore North, Bangalore Urban, Karnataka, 560084, India"
8861892eb7fffff,8861892eb7fffff,1,2.129961,"Muneshwara Nagar, East Zone, Bengaluru, Bangalore North, Bangalore Urban, Karnataka, 560084, India"
8860145a2dfffff,8860145a2dfffff,1,1.442236,"K H Ranganath Colony, Rayapuram Ward, West Zone, Bengaluru, Bangalore North, Bangalore Urban, Karnataka, 560026, India"
8861892e03fffff,8861892e03fffff,0,1.338503,"Mapple Heights Apartments, Vijnana Nagar, Mahadevapura Zone, Bengaluru, Bangalore East, Bangalore Urban, Karnataka, 560093, India"
886189246bfffff,886189246bfffff,0,0.965623,"AECS Layout, A block, Singasandra, Bommanahalli Zone, Bengaluru, Anekal, Bangalore Urban, Karnataka, - 560068, India"
88618924b3fffff,88618924b3fffff,0,0.681751,"AGS Layout, Uttarahalli, Bommanahalli Zone, Bengaluru, Bangalore South, Bangalore Urban, Karnataka, 560061, India"


In [7]:
map_centre = (12.9792,77.5916)

map1 = folium.Map(location = map_centre, zoom_start = 11)

bins = [
    df_scores_1['score'].min(),
    df_scores_1['score'].quantile(0.50),
    df_scores_1['score'].quantile(0.85),
    df_scores_1['score'].quantile(0.95),
    df_scores_1['score'].quantile(0.99),
    df_scores_1['score'].max(),
]

choropleth = folium.Choropleth(
    geo_data = df_scores_1,
    data = df_scores_1['score'],
    key_on = 'id',
    fill_color = 'YlGnBu',
    fill_opacity = 0.8,
    nan_fill_opacity = 0.0,
    line_opacity = 0.9,
    legend_name = 'Score (higher is better)',
    bins = bins,
    highlight = True,
)

popup = folium.GeoJsonPopup(
    fields = ['id', 'address', 'score'],
    aliases = ['Hex ID', 'Address', 'Score'],
).add_to(choropleth.geojson)

map1.add_child(choropleth)

map1 # Display map

## Scenario 2: College Bar

Here, our primary targets are college students and young working professionals - so look for areas near colleges or offices, that do not already have a lot of competition.

In [8]:
# Assign relative importance of different venues
weights = {
    'pop_total': 10,
    'cost_sqft': -15,
    'Nightlife Spot': -15.0,
    'Residence': 5,
    'Office': 10,
    'Shopping Mall': 10,
    'Movie Theater': 10,
    'College & University': 15,
    'Arts & Entertainment': 7,
}

In [9]:
scores = pd.Series(dtype = 'float')
counts = []
# For each cell, we will add the cells own score plus 20% of the score of neighbouring cells.
for id, row in df_features.iterrows():
    score = scoreLocation(row, weights)
    neighbours = list(h3.k_ring(id, 1))
    neighbours.remove(id) # Remove root cell - no double counting
    count = 0 # Number of neighbours (check)
    for n in neighbours:
        if n in df_features.index:
            score += (0.2 * scoreLocation(df_features.loc[n], weights))
            count +=1
    scores[id] = score
    counts.append(count)

df_scores_2 = df.copy().set_index('id', drop = False)
df_scores_2.insert(2, 'score', scores)
df_scores_2 = df_scores_2.sort_values('score', ascending = False)

# Display top 10 locations
pd.options.display.max_colwidth = -1
df_scores_2[['id', 'cluster', 'score', 'address']].head(10)

<ipython-input-9-712007a57a6f>:21: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.options.display.max_colwidth = -1


,id,cluster,score,address
id,,,,
8860145b1dfffff,8860145b1dfffff,3,11.474030,"Hospital-ayurvedic Homeo Clinic, 2nd Main Road, Prakash Nagar Ward, West Zone, Bengaluru, Bangalore North, Bangalore Urban, Karnataka, 560021, India"
88618925d9fffff,88618925d9fffff,2,8.634638,"MICO Layout, BTM Layout Ward, South Zone, Bengaluru, Bangalore South, Bangalore Urban, Karnataka, 560069, India"
8861892185fffff,8861892185fffff,3,8.518986,"Kadugodi, Mahadevapura Zone, Sheegehalli, Bangalore East, Bangalore Urban, Karnataka, 56066, India"
8860145b0bfffff,8860145b0bfffff,3,8.235069,"Ramachandra Pura, Okalipuram Ward, West Zone, Bengaluru, Bangalore North, Bangalore Urban, Karnataka, 560 020, India"
88618925d5fffff,88618925d5fffff,3,7.799838,"Bismillah Nagar, Gurappanapalya Ward, South Zone, Bengaluru, Bangalore South, Bangalore Urban, Karnataka, 560029, India"
88618925dbfffff,88618925dbfffff,3,7.687766,"JP Nagar 2nd Phase, South Zone, Bengaluru, Bangalore South, Bangalore Urban, Karnataka, 560069, India"
8861892cd7fffff,8861892cd7fffff,0,7.537133,"4th Cross Road, Anandanagar, Hebbala Ward, East Zone, Bengaluru, Bangalore North, Bangalore Urban, Karnataka, 560032, India"
8860169625fffff,8860169625fffff,1,7.211961,"Chowdeswari Ward, Yelahanka Zone, Bengaluru, Bangalore North, Bangalore Urban, Karnataka, 560064, India"
88618925d1fffff,88618925d1fffff,2,7.190631,"Sahakari Vidyakendra AHPS Jayanagar, East End B Main Road, NAL Layout, Jayanagar East Ward, South Zone, Bengaluru, Bangalore South, Bangalore Urban, Karnataka, 560069, India"


In [10]:
map_centre = (12.9792,77.5916)

map2 = folium.Map(location = map_centre, zoom_start = 11)

bins = [
    df_scores_2['score'].min(),
    df_scores_2['score'].quantile(0.50),
    df_scores_2['score'].quantile(0.85),
    df_scores_2['score'].quantile(0.95),
    df_scores_2['score'].quantile(0.99),
    df_scores_2['score'].max(),
]

choropleth = folium.Choropleth(
    geo_data = df_scores_2,
    data = df_scores_2['score'],
    key_on = 'id',
    fill_color = 'YlGnBu',
    fill_opacity = 0.8,
    nan_fill_opacity = 0.0,
    line_opacity = 0.9,
    legend_name = 'Score (higher is better)',
    bins = bins,
    highlight = True,
)

popup = folium.GeoJsonPopup(
    fields = ['id', 'address', 'score'],
    aliases = ['Hex ID', 'Address', 'Score'],
).add_to(choropleth.geojson)

map2.add_child(choropleth)

map2 # Display map